Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [89]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [90]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [19]:
pip install tfidf

Note: you may need to restart the kernel to use updated packages.


In [91]:
#Dataset
from sklearn.datasets import fetch_20newsgroups

categories =['alt.atheism',
            'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)


In [198]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [94]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [95]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [96]:
grid_search.best_score_

0.8879813302217037

In [97]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [100]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [101]:
pwd

'/Users/jorge/Unit 4 DS 10/DS-Unit-4-Sprint-1-NLP/module3-document-classification'

In [102]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [103]:
train.tail()

,id,description,ratingCategory
4082,3342,\nWhat lies beneath the surface of Dewar’s? He...,1
4083,3130,\nAfter 6 to 7 years of maturation in bourbon ...,1
4084,2811,"\nBright, delicate, and approachable. While no...",1
4085,478,\nI’m calling this the pitmaster’s dram: the n...,1
4086,1209,"\nSpicy sultanas, greengage plums, toffee, and...",1


In [104]:
train.describe(exclude='number')

,description
count,4087
unique,4087
top,"\nThis is a single cask (#729) bottling, disti..."
freq,1


In [105]:
train.dtypes

id                 int64
description       object
ratingCategory     int64
dtype: object

In [106]:
train['description'][0]

'\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

### Define Pipeline Components

In [197]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [199]:
help(TfidfVectorizer)

Help on class TfidfVectorizer in module sklearn.feature_extraction.text:

class TfidfVectorizer(CountVectorizer)
 |  TfidfVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, analyzer='word', stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.float64'>, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)
 |  
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  


In [136]:
tokenize = vect.build_tokenizer()
stop_words = vect.get_stop_words()

In [137]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 10,
 'vect__max_df': 0.75,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

In [138]:
grid_search.best_score_

0.8879813302217037

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [202]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df' : (3, 5),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1, scoring='accuracy')
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   17.2s
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:   29.2s finished
/Users/jorge/opt/anaconda3/envs/U4-S1-NLP-DS10/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (3, 5), 'clf__max_depth': (5, 10, 15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [214]:
grid_search.best_score_

0.9066511085180864

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this compeition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [215]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [216]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [217]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [218]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

You're trying to achieve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [219]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [220]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [221]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [222]:
print(pipe)

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])


In [213]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 10.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': [0.9, 0.95, 1.0], 'clf__n_estimators': [5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [223]:
grid_search.best_score_

0.9066511085180864

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [225]:
from sklearn.svm import SVC, LinearSVC, NuSVC
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)
vect = TfidfVectorizer(ngram_range=(1,3))
lsi = Pipeline([('vect', vect), ('svd', svd)])

clf = LinearSVC()

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [226]:
tokenize = vect.build_tokenizer()
stop_words = vect.get_stop_words()

In [227]:
grid_search.best_score_

0.9066511085180864

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [229]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95],
    'lsi__vect__min_df':[3, 5, 10],
    'lsi__vect__stop_words': [None, 'english', stop_words],
    'lsi__vect__tokenizer': [None, tokenize],
    'clf__loss':('hinge', 'squared_hinge'),
    'clf__C':[0.1, 0.3, 0.5, 0.8, 1]
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=10)
grid_search.fit(train['description'], train['ratingCategory'])


Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   33.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   40.4s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   49.5s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   57.4s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm=...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': [0.9, 0.95], 'lsi__vect__min_df': [3, 5, 10], 'lsi__vect__stop_words': [None, 'english', None], 'lsi__vect__tokenizer': [None, <function VectorizerMixin.build_tokenizer.<locals>.<lambda> at 0x144960400>], 'clf__loss': ('hinge', 'squared_hinge'), 'clf__C': [0.1, 0.3, 0.5, 0.8, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=N

In [230]:
grid_search.best_params_

{'clf__C': 0.8,
 'clf__loss': 'squared_hinge',
 'lsi__svd__n_components': 250,
 'lsi__vect__max_df': 0.9,
 'lsi__vect__min_df': 10,
 'lsi__vect__stop_words': None,
 'lsi__vect__tokenizer': None}

In [245]:
grid_search.best_score_

0.7516515781747003

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [232]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [233]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [234]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [235]:
subNumber = 4

In [236]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

In [278]:
nlp = spacy.load('en_core_web_lg')

In [279]:
doc = nlp(train.description[1])
print(type(doc))
doc

<class 'spacy.tokens.doc.Doc'>



An uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness.  (Wyoming only)

In [280]:
vector_alpha = doc.vector
print(type(vector_alpha))
print(len(vector_alpha))

<class 'numpy.ndarray'>
300


In [281]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [282]:
X = get_word_vectors(train.description)

len(X) == len(train.description)

True

In [ ]:
len(X)

In [ ]:
rfc.fit(X, train.trainingCategory)

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [237]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [238]:
doc = nlp("Two bananas in pyjamas")

In [239]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [240]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [247]:
X = get_word_vectors(train['description'])

len(X) == len(data.data)

False

In [248]:
len(X)

4087

In [299]:
rfc.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [284]:
rfc.score(X, train['ratingCategory']

0.991436261316369

In [285]:
rfc.predict(X)

array([1, 0, 1, ..., 1, 1, 1])

In [297]:
test.head(3)

,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...


In [300]:
test_embeddings = get_word_vectors(test.description)
len(test_embeddings) ==len(test.description)

True

In [301]:
prediction = rfc.predict(test_embeddings)
len(prediction)

1022

In [294]:
submission5 = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission5['ratingCategory'] = submission5['ratingCategory'].astype('int64')

In [295]:
submission5.head(3)

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1


In [296]:
submission5.to_csv('./data/submission5.csv', index=False)

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on